In [13]:
# ── 1. Imports ──────────────────────────────────────
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from IPython.display import Image, display
import pandas as pd
from tqdm import tqdm
import shutil
import os

In [14]:

# ── 2. Clean and Setup ──────────────────────────────
persist_dir = "/tmp/chroma_store_test"

if os.path.exists(persist_dir):
    print("🧹 Removing old vectorstore...")
    shutil.rmtree(persist_dir)


In [15]:
# ── 3. Load Dataset ─────────────────────────────────
print("📦 Loading dataset...")
product = pd.read_csv("products_cleaned.csv")
product

📦 Loading dataset...


,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,isBestSeller,boughtInLastMonth,id,category_name
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,False,2000,104,Suitcases
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,False,1000,104,Suitcases
2,B07XSCCZYG,Platinum Elite Softside Expandable Checked Lug...,https://m.media-amazon.com/images/I/71EA35zvJB...,https://www.amazon.com/dp/B07XSCCZYG,4.6,0,365.49,429.99,False,300,104,Suitcases
3,B08MVFKGJM,Freeform Hardside Expandable with Double Spinn...,https://m.media-amazon.com/images/I/91k6NYLQyI...,https://www.amazon.com/dp/B08MVFKGJM,4.6,0,291.59,354.37,False,400,104,Suitcases
4,B01DJLKZBA,Winfield 2 Hardside Expandable Luggage with Sp...,https://m.media-amazon.com/images/I/61NJoaZcP9...,https://www.amazon.com/dp/B01DJLKZBA,4.5,0,174.99,309.99,False,400,104,Suitcases
...,...,...,...,...,...,...,...,...,...,...,...,...
1426332,B00R3LIKCO,American Flag Patriotic USA Classic 5 Panel Me...,https://m.media-amazon.com/images/I/71PDJFz6AA...,https://www.amazon.com/dp/B00R3LIKCO,4.2,0,14.95,0.00,False,0,112,Men's Accessories
1426333,B098BQ7ZQ3,Men's Baseball Cap - H2O-DRI Line Up Curved Br...,https://m.media-amazon.com/images/I/812Tycexs4...,https://www.amazon.com/dp/B098BQ7ZQ3,4.4,0,33.99,0.00,False,0,112,Men's Accessories
1426334,B07X1MVNT1,[4 Pack] Adjustable Eyeglasses and Sunglasses ...,https://m.media-amazon.com/images/I/61vvYW1S9J...,https://www.amazon.com/dp/B07X1MVNT1,3.6,0,8.54,0.00,False,0,112,Men's Accessories
1426335,B08XLBG8V9,Ax2002 Aviator Sunglasses,https://m.media-amazon.com/images/I/51+yjD4F1x...,https://www.amazon.com/dp/B08XLBG8V9,4.5,0,54.36,57.39,False,0,112,Men's Accessories


In [16]:

# Ensure 'image_url' column exists
if "imgUrl" not in product.columns:
    raise ValueError("❌ Your dataset is missing the 'image_url' column.")


In [17]:
# ── 4. Combine Text for Embedding ───────────────────
product["text"] = (
    product["title"].fillna("") + " " +
    product["category_name"].fillna("") + " $" +
    product["price"].fillna("").astype(str) + " stars " +
    product["stars"].fillna("").astype(str)
)

product.dropna(subset=["text"], inplace=True)
product = product[product["text"].str.strip().astype(bool)]


In [18]:
# ── 5. Create LangChain Documents ───────────────────
print("📄 Creating documents...")
documents = [
    Document(
        page_content=row["text"],
        metadata={
            "id": int(row["id"]),
            "title": row["title"],
            "imgUrl": row.get("imgUrl", "")
        }
    )
    for _, row in tqdm(product.iterrows(), total=len(product))
]


📄 Creating documents...


100%|██████████| 1426337/1426337 [00:36<00:00, 39023.92it/s]


In [19]:
# ── 6. Load Embedding Model ─────────────────────────
print("🧠 Loading embedding model...")
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


🧠 Loading embedding model...


In [20]:
# ── 7. Create Chroma Vectorstore ────────────────────
import shutil

if os.path.exists(persist_dir):
    print("🧹 Removing old vectorstore due to permission error...")
    shutil.rmtree(persist_dir)

os.makedirs(persist_dir, exist_ok=True)

print("💾 Saving to vectorstore...")
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_dir
)
vectorstore.persist()
print("✅ Vectorstore created successfully.")


💾 Saving to vectorstore...
✅ Vectorstore created successfully.


/var/folders/32/_dxyx4ls74q6tdn_wtgm2s480000gn/T/ipykernel_17294/3711669321.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [21]:
# ── 8. Search & Display Results ─────────────────────
def search_and_display(query: str, k: int = 5):
    print(f"\n🔍 Searching for: \"{query}\"\n")
    results = vectorstore.similarity_search(query, k=k)

    for i, res in enumerate(results):
        print(f"Result {i+1}")
        print("📌 Title:", res.metadata.get("title", "N/A"))
        print("📝 Text:", res.page_content)
        image_url = res.metadata.get("imgUrl")
        if image_url:
            try:
                display(Image(url=image_url, width=250))
            except:
                print("⚠️ Couldn't load image.")
        else:
            print("❌ No image URL provided.")
        print("-" * 50)


In [22]:
# ── 9. Example Test Query ───────────────────────────
search_and_display("wireless bluetooth headphones with noise cancellation")


🔍 Searching for: "wireless bluetooth headphones with noise cancellation"

Result 1
📌 Title: Wireless Headphones Over Ear for Active Noise Cancelling Headphones Bluetooth Headset with Microphone Dance Music DJ Headphones Designed for Game Calls and Music, Foldable to Save Space etc (Black)
📝 Text: Wireless Headphones Over Ear for Active Noise Cancelling Headphones Bluetooth Headset with Microphone Dance Music DJ Headphones Designed for Game Calls and Music, Foldable to Save Space etc (Black) Headphones & Earbuds $49.99 stars 0.0


--------------------------------------------------
Result 2
📌 Title: Bluetooth Headset Noise Canceling for Cellphones Wireless Single Ear Bluetooth Earpiece with Microphone for Trucker（White）
📝 Text: Bluetooth Headset Noise Canceling for Cellphones Wireless Single Ear Bluetooth Earpiece with Microphone for Trucker（White） Wearable Technology $10.99 stars 2.8


--------------------------------------------------
Result 3
📌 Title: AmazonCommercial Wireless Noise Cancelling Bluetooth Headphones
📝 Text: AmazonCommercial Wireless Noise Cancelling Bluetooth Headphones Headphones & Earbuds $143.15 stars 3.9


--------------------------------------------------
Result 4
📌 Title: Bluetooth Headset Noise Canceling for Cellphones Wireless Single Ear Bluetooth Earpiece with Microphone for Trucker (Dark Black)
📝 Text: Bluetooth Headset Noise Canceling for Cellphones Wireless Single Ear Bluetooth Earpiece with Microphone for Trucker (Dark Black) Wearable Technology $9.99 stars 3.7


--------------------------------------------------
Result 5
📌 Title: Bluetooth Active Noise Cancellation Headphone with Build in Microphone
📝 Text: Bluetooth Active Noise Cancellation Headphone with Build in Microphone Headphones & Earbuds $49.95 stars 4.0


--------------------------------------------------


In [23]:
search_and_display("mens formal shoes")


🔍 Searching for: "mens formal shoes"

Result 1
📌 Title: Mens Dress Shoes Oxford Style Formal Classic Lace Up Business Casual Uniform Modern Work Soft Wedding Shoes Black Brown US6-15
📝 Text: Mens Dress Shoes Oxford Style Formal Classic Lace Up Business Casual Uniform Modern Work Soft Wedding Shoes Black Brown US6-15 Men's Shoes $29.99 stars 4.1


--------------------------------------------------
Result 2
📌 Title: Men's Dress Shoes Comfortable Breathable Casual Flat Plain Sneakers Classic Modern Formal Oxfords Applicable to Business,Work,Party,Wedding,Daily Walk
📝 Text: Men's Dress Shoes Comfortable Breathable Casual Flat Plain Sneakers Classic Modern Formal Oxfords Applicable to Business,Work,Party,Wedding,Daily Walk Men's Shoes $49.99 stars 4.2


--------------------------------------------------
Result 3
📌 Title: Men's Casual Sneakers Dress Shoes - Comfort Dress Sneakers Mesh Knit Lightweight Breathable Formal Oxford Business Shoes
📝 Text: Men's Casual Sneakers Dress Shoes - Comfort Dress Sneakers Mesh Knit Lightweight Breathable Formal Oxford Business Shoes Men's Shoes $47.99 stars 4.2


--------------------------------------------------
Result 4
📌 Title: Mens Sneakers. Fashion Sneakers. Lace-Up Or Slip-On Mens Casual Shoes. Comfortable Leather Shoes.
📝 Text: Mens Sneakers. Fashion Sneakers. Lace-Up Or Slip-On Mens Casual Shoes. Comfortable Leather Shoes. Men's Shoes $89.95 stars 4.3


--------------------------------------------------
Result 5
📌 Title: Men's Oxfords Formal Shoes Men Business Casual Shoes Lace-up Fashion Dress Shoes for Men
📝 Text: Men's Oxfords Formal Shoes Men Business Casual Shoes Lace-up Fashion Dress Shoes for Men Men's Shoes $45.99 stars 4.3


--------------------------------------------------


In [24]:
search_and_display("ladies shoes")


🔍 Searching for: "ladies shoes"

Result 1
📌 Title: Women's Blade Running Shoes Ladies Fashion Sneakers Slip on Shoes Tennis Walking Athletic Shoes
📝 Text: Women's Blade Running Shoes Ladies Fashion Sneakers Slip on Shoes Tennis Walking Athletic Shoes Women's Shoes $37.99 stars 4.4


--------------------------------------------------
Result 2
📌 Title: Womens Fashion Tennis Sneakers Non-Slip Lace Up Walking Shoes Mens Comfort Running Shoes Ladies Skate Shoes Black with Skull Painted 7
📝 Text: Womens Fashion Tennis Sneakers Non-Slip Lace Up Walking Shoes Mens Comfort Running Shoes Ladies Skate Shoes Black with Skull Painted 7 Men's Shoes $36.99 stars 5.0


--------------------------------------------------
Result 3
📌 Title: Women Fashion Sneakers Floral Print Lace-up Casual Walking Shoes
📝 Text: Women Fashion Sneakers Floral Print Lace-up Casual Walking Shoes Women's Shoes $44.99 stars 4.4


--------------------------------------------------
Result 4
📌 Title: Womens Slip On Shoes Fashion Sneakers Comfortable Walking Flats Casual Shoes
📝 Text: Womens Slip On Shoes Fashion Sneakers Comfortable Walking Flats Casual Shoes Women's Shoes $32.99 stars 4.3


--------------------------------------------------
Result 5
📌 Title: Women’s Fashion Sneakers Slip On Shoes Women Sneakers Canvas Shoes Womens Tennis Comfortable Walking Flats for Womens Casual Shoes
📝 Text: Women’s Fashion Sneakers Slip On Shoes Women Sneakers Canvas Shoes Womens Tennis Comfortable Walking Flats for Womens Casual Shoes Women's Shoes $39.9 stars 4.3


--------------------------------------------------


In [25]:
search_and_display("mens wears fashionable")


🔍 Searching for: "mens wears fashionable"

Result 1
📌 Title: mens Men Modern
📝 Text: mens Men Modern Men's Clothing $169.0 stars 4.1


--------------------------------------------------
Result 2
📌 Title: Brand Men's COTTON Dress Pants PURPLE INDIGO Flat Front Mens Trousers
📝 Text: Brand Men's COTTON Dress Pants PURPLE INDIGO Flat Front Mens Trousers Men's Clothing $49.3 stars 4.3


--------------------------------------------------
Result 3
📌 Title: mens Stretch Modern-fit Flat-front dress pants, Black, 34W x 32L US
📝 Text: mens Stretch Modern-fit Flat-front dress pants, Black, 34W x 32L US Men's Clothing $46.0 stars 4.1


--------------------------------------------------
Result 4
📌 Title: mens Shapewear Form Tank
📝 Text: mens Shapewear Form Tank Men's Clothing $31.5 stars 4.3


--------------------------------------------------
Result 5
📌 Title: Mens Stretch Skinny Flat-Front Fashion Style Comfy Solid Casual Slim Fit Business Dress Chinos Pants
📝 Text: Mens Stretch Skinny Flat-Front Fashion Style Comfy Solid Casual Slim Fit Business Dress Chinos Pants Men's Clothing $27.99 stars 4.3


--------------------------------------------------
